Notebook which will:
- Obtain alert photometry for named SN and convert to AMPEL LightCurve object.
- Run T2BrightSNProb / T2SNGuess for each state.
- Print eval.

In [ ]:
import pandas as pd
import requests, os, time
from requests.auth import HTTPBasicAuth
import numpy as np
import matplotlib.pyplot as plt
#import lcdata
from astropy.table import Table
from IPython.core.display import Image, display

In [ ]:
%load_ext ampel_quick_import
%qi AmpelLogger ZTFIdMapper ZiDataPointShaper T1Document LightCurve T2BrightSNProb 

In [ ]:
name = "ZTF21aclhsxb"
desy_archive_user =   # Add username 
desy_archive_pw =    # Add pw

In [ ]:
# Initalize t2 unit
logger = AmpelLogger.get_logger()
t2snguess = T2BrightSNProb( logger=logger)
t2snguess.post_init()
# Shaper which knows structure of ZTF alerts
shaper = ZiDataPointShaper(logger=logger)

In [ ]:
# Get *all* alerts for transient
endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/object/{}/alerts?with_history=true&with_cutouts=false".format(name)
auth = HTTPBasicAuth(desy_archive_user, desy_archive_pw)
response = requests.get(endpoint, auth=auth )

In [ ]:
if not response.ok:
    print('... failed to get alert')
alerts = response.json()
if alerts is None:
    print(' ... no alert content')

In [ ]:
print("Found {} alerts for {}".format(len(alerts), name ) )

Loop through alerts, create LightCurve object and run T2BrightSNProb

In [ ]:
summary = []
for alert in alerts:
    
    # Create standardized LightCurve object
    pps = [alert['candidate']]
    pps.extend( [prv_cand for prv_cand in alert['prv_candidates'] ] )
    
    # The following loop is _likely_ due to an inconsistency in the alert archive with the shaper
    # and can hopefully be removed soon
    for pp in pps:
        if "magpsf" in pp.keys():
            pp["candid"] = 999

    stockId = ZTFIdMapper.to_ampel_id(name)
    dps = shaper.process( pps, stockId)
    t1d = T1Document(stock=stockId, link=0)
    lc = LightCurve.build(t1d, dps)
    
    # Concent
    jds = lc.get_values("jd")
    if jds is None:
        continue
    
    t2out = t2snguess.process(lc)
    print(t2out)
    
    if t2out['success']:
        summary.append( [max(jds),len(jds),t2out['success'], t2out['SNGuess'], t2out['SNGuessBool'] ])
    else:
        summary.append( [max(jds),len(jds),t2out['success'] ])
        

In [ ]:
summary